In [16]:
# ======================================================
# 🎓 University Course Material Recommender (Final NLP Version)
# ======================================================

import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import spacy
import nltk

# Download necessary NLTK data (only once)
nltk.download('wordnet')
nltk.download('omw-1.4')

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Load dataset
df = pd.read_csv("course_material.csv")

# Combine all columns into one searchable text
df["combined_text"] = df.apply(
    lambda row: " ".join(str(x) for x in row.values if pd.notnull(x)),
    axis=1
)

# Initialize TF-IDF
vectorizer = TfidfVectorizer(stop_words="english")
X = vectorizer.fit_transform(df["combined_text"])

# Common material-type words
material_keywords = ["book", "slide", "note", "lab", "video", "manual", "sheet", "handout"]

# ------------------------------------------------------
# 🧠 Preprocessing (Lemmatization + Synonym Expansion)
# ------------------------------------------------------
def preprocess_text(text):
    text = text.lower()
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    
    expanded = set(lemmas)
    for word in lemmas:
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                expanded.add(lemma.name().replace("_", " "))
    
    return " ".join(expanded)

# ------------------------------------------------------
# 🔍 Main Recommendation Function
# ------------------------------------------------------
def recommend_materials(query, top_n=5, threshold=0.1):
    processed_query = preprocess_text(query)
    match = re.search(r"[A-Z]{3}\s*\d{4}", query.upper())
    filtered_df = df.copy()
    filtered_X = X

    # 🔎 Course code filtering if mentioned
    if match:
        course_code = match.group(0).replace(" ", "")
        filtered_df = df[df["course_code"].str.replace(" ", "") == course_code]
        if not filtered_df.empty:
            filtered_X = vectorizer.transform(filtered_df["combined_text"])

    # Compute similarity
    query_vec = vectorizer.transform([processed_query])
    similarity = cosine_similarity(query_vec, filtered_X).flatten()

    results = []
    query_lower = query.lower()

    for i, idx in enumerate(filtered_df.index):
        score = similarity[i]

        # Fuzzy matching for material type
        for kw in material_keywords:
            if any(kw in word for word in query_lower.split()) and kw in df.loc[idx, "material_type"].lower():
                score += 0.2

        if score > threshold:
            result_text = (
                f"\n✅ Found material:\n"
                f"📘 Course: {df.loc[idx, 'course_name']} ({df.loc[idx, 'course_code']})\n"
                f"📂 Type: {df.loc[idx, 'material_type']}\n"
                f"📄 Title: {df.loc[idx, 'material_title']}\n"
                f"🔗 Link: {df.loc[idx, 'material_link']}\n"
                f"📝 Description: {df.loc[idx, 'description']}\n"
            )
            results.append((score, result_text))

    results = sorted(results, key=lambda x: x[0], reverse=True)

    # 🧩 Fallback: show all materials for that course if specific not found
    if not results and match:
        fallback = df[df["course_code"].str.replace(" ", "") == course_code]
        if not fallback.empty:
            return "\n".join([
                f"\n📘 Course: {row['course_name']} ({row['course_code']})\n"
                f"📂 Type: {row['material_type']}\n"
                f"📄 Title: {row['material_title']}\n"
                f"🔗 Link: {row['material_link']}\n"
                f"📝 Description: {row['description']}\n"
                for _, row in fallback.iterrows()
            ])

    # Nothing at all found
    if not results:
        return "❌ Sorry, I couldn't find any relevant material for your query."

    return "\n".join([r[1] for r in results[:top_n]])


# ------------------------------------------------------
# 🧪 Test Queries
# ------------------------------------------------------
queries = [
    "Operating System Slide",
]

for q in queries:
    print(f"\n🔍 Query: {q}")
    print(recommend_materials(q))


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nasim\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Nasim\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!



🔍 Query: Operating System Slide

✅ Found material:
📘 Course: Operating Systems (CSE 3201)
📂 Type: Slide
📄 Title: Slide on Operating System
🔗 Link: https://drive.google.com/drive/folders/1DaldLOmRxvblq60hn8Ln8maRTR5YgqD5
📝 Description: operating system slides provided by teacher


✅ Found material:
📘 Course: Computer Interfacing and Embedded Systems (CSE 3105)
📂 Type: Slide
📄 Title: Slide on CIES
🔗 Link: https://drive.google.com/drive/folders/1NqMPdtoDcPoLzjKjgI8rOwK7wngdv2R1
📝 Description: Slide provided by teacher


✅ Found material:
📘 Course: Software Engineering (CSE 3205)
📂 Type: Slide
📄 Title: Slide on Software Engineering 
🔗 Link: https://drive.google.com/drive/folders/1huvCY1ky56LxjE5lzeiDHwGSgfGWY1aV
📝 Description: slide provide by teacher on software engineering


✅ Found material:
📘 Course: Structured Programming (CSE 1101)
📂 Type: Slide
📄 Title: Slide on C Programming
🔗 Link: https://drive.google.com/drive/folders/1L0zNT3RjlYrqAfEBUthnRLJxExi4uojd
📝 Description: intro to pr